In [1]:
!pip install trl datasets bitsandbytes accelerate peft tensorboardX -qqU

## DPO Preparations

In [2]:
import pandas as pd
import numpy as np

In [3]:
path = '/home/jovyan/'

In [4]:
preferences_df = pd.read_csv(path + 'dpo_chosen_rejected_texts.csv')

In [5]:
## на завтра
## убрать из промпта тесты
## проверить эту гипотезу
## + проверить, оставив только решения и условие

In [6]:
preferences_df.sample(2)

,Unnamed: 0.1,Unnamed: 0,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt,conversations,conversations2
51,51,51,94,6,"Напишите функцию, определяющую дизайнера, кото...","[""['Круглов', 'Квадратов', 'Иванов', 'Марков',...","['Гуськов', 'Краков', 'Волков', 'Стариков', 'П...","def designer (designers, sizes, towns):\n l...","def designer (designers, sizes, towns):\n s...",Ошибка при использовании функции min(). Функци...,"['open', 'open', 'closed', 'closed', 'closed',...","<Task description>: Напишите функцию, определя...","[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC..."
15,15,15,28,2,"Реализуйте программу, которая определит количе...",['ПРОЕКТ 1\nПРОЕКТ_2\nпроект 3\nПроект_4\nинте...,"['1', '0', '1', '1', '1']",result = 0\n\nwhile True:\n info = input()\...,result = 0\n\nwhile True:\n info = input()\...,Обратите внимание на ошибку в табуляции выраже...,"['open', 'open', 'closed', 'closed', 'closed']","<Task description>: Реализуйте программу, кото...","[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC..."


In [7]:
preferences_df.shape

(347, 14)

In [8]:
prompt_list = []
chosen_list = []
rejected_list = []
messages_list = [] # = chosen

for index, row in preferences_df.iterrows():
  prompt = row['prompt']
  prompt_list.append(prompt)
  chosen = row['conversations']
  chosen_list.append(chosen)
  messages_list.append(chosen)
  rejected = row['conversations2']
  rejected_list.append(rejected)

In [9]:
preferences_dpo_df = pd.DataFrame({'prompt': prompt_list,
                                   'chosen': chosen_list,
                                   'rejected': rejected_list,
                                   'messages': messages_list,})

In [10]:
preferences_dpo_df.sample(2)

,prompt,chosen,rejected,messages
253,<Task description>: Роман изучает свой отчет п...,"[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC..."
96,"<Task description>: Напишите функцию, определя...","[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC...","[{'role': 'system', 'content': '\n\n###INSTRUC..."


In [11]:
preferences_dpo_df.to_csv('preferences_dpo_final_df.csv')

## DPO Train

In [12]:
from datasets import Dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline, TrainingArguments
from peft import LoraConfig, get_peft_model
import peft
import transformers

import torch

In [13]:
train_dataset = Dataset.from_pandas(preferences_dpo_df.reset_index(drop=True))

In [14]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'messages'],
    num_rows: 347
})

In [15]:
import torch
torch.cuda.empty_cache()

In [16]:
!nvidia-smi

Thu Oct 17 17:24:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:B3:00.0 Off |                    0 |
| N/A   37C    P0              80W / 700W |      2MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [17]:
model_name = 'Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map='balanced',
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
tokenizer.truncation_side = 'left'

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
from accelerate import Accelerator

accelerator = Accelerator()
model, tokenizer, train_dataset = accelerator.prepare(model, tokenizer, train_dataset)

In [19]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

system_prompt = "You are a helpful assistant, just answer user's questions."
prompt = 'Кто ты?'

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
  ]
answer = pipe(messages, max_length=6048)[0]['generated_text'][-1]['content']
answer

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'Я — цифровой помощник, созданный для того, чтобы помогать пользователям, отвечая на их вопросы и выполняя различные задачи.'

In [20]:
for name, param in model.named_parameters():
  print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [21]:
peft_config = LoraConfig(
    use_dora=True,
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout=0.2,
    bias='none',
    layers_to_transform=[38, 39],
    task_type="CAUSAL_LM",
)

#peft_model = get_peft_model(model, peft_config).to('сuda:0')

In [22]:
#peft_model.device

In [23]:
from numpy import percentile

prompt_length = int(percentile([len(tokenizer(x)["input_ids"]) for x in train_dataset["prompt"]], 95))
max_seq_length_chosen = int(percentile([len(tokenizer(x["prompt"] + x["chosen"])["input_ids"]) for x in train_dataset], 95))
max_seq_length_rejected = int(percentile([len(tokenizer(x["prompt"] + x["rejected"])["input_ids"]) for x in train_dataset], 95))
max_seq_length = max(max_seq_length_chosen, max_seq_length_rejected)

train_dataset = train_dataset.filter(lambda x: len(tokenizer(x["prompt"] + x["chosen"])["input_ids"]) <= max_seq_length)
print(f"len(train_dataset): {len(train_dataset)}")

prompt_length = ((prompt_length + 1) // 2) * 2
max_seq_length = ((max_seq_length + 1) // 2) * 2
print(f"p95 prompt length: {prompt_length}")
print(f"p95 prompt + chosen length: {max_seq_length}")

Filter:   0%|          | 0/347 [00:00<?, ? examples/s]

len(train_dataset): 331
p95 prompt length: 1466
p95 prompt + chosen length: 4514


In [24]:
prompt_length = 1466
max_seq_length = 4514

In [25]:
training_args = DPOConfig(
    output_dir="DPO_res",
    #num_train_epochs=1,
    max_steps=20, ################## TUNE
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=5e-5,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=1,
    save_steps=10,
    save_total_limit=10,
    bf16=True,
    report_to="tensorboard",
    use_cpu=False,
    remove_unused_columns=False,
    max_length=max_seq_length,
    max_prompt_length=prompt_length,
)

dpo_args = {
    "beta": 0.1,
    "loss_type": "sigmoid"
}

In [26]:
trainer = DPOTrainer(
    model,
    ref_model=None,
    peft_config=peft_config,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    beta=dpo_args["beta"],
    #loss_type=dpo_args["loss_type"],
)

Tokenizing train dataset:   0%|          | 0/331 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [27]:
model.gradient_checkpointing_enable()

In [28]:
accelerator.wait_for_everyone()
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.696500
4,0.688300
5,0.672100
6,0.646200
7,0.602000
8,0.573200
9,0.524600
10,0.495000


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=20, training_loss=0.4962386041879654, metrics={'train_runtime': 464.0325, 'train_samples_per_second': 0.345, 'train_steps_per_second': 0.043, 'total_flos': 0.0, 'train_loss': 0.4962386041879654, 'epoch': 0.4819277108433735})

## INFERENCE

In [29]:
!pip install openpyxl -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
final_test_df = pd.read_csv(path + 'test_database.csv')
test_solutions = pd.read_excel(path + 'test/solutions.xlsx')

In [31]:
final_test_df.shape, test_solutions.shape

((119, 9), (325, 5))

In [32]:
test_solutions.sample(2)

,id,task_id,student_solution,author_comment,author_comment_embedding
234,584,36,time = input()\n\n# ваш код ниже\n\nprint(f'В ...,Ваше предсказание,NaN
110,279,17,"room = input()\n\nresidences = []\nfor key, va...",Ваше предсказание,NaN


In [33]:
final_test_df.sample(2)

,id,number,task_id,type,input,output,level,description,author_solution
73,141,3,24,closed,антитезы,Сегодня мы будем анализировать антитезы в тексте!,B,"Реализуйте программу, которая напечатает анали...","ton = input()\n\n# ваш код ниже\n\nprint(f""Сег..."
91,198,0,36,open,неделю,В агентстве «Шедеврус» мы делаем дизайн за нед...,B,Реализуйте следующую программу: \n\nВ переменн...,time = input()\n\n# ваш код ниже\n\nprint(f'В ...


In [34]:
new_test_df = {'id': [], 'task_id': [], 'description': [],
          'input': [], 'output': [], 'student_solution': [],
          'author_solution': [], 'author_comment': [], 'type': []}

In [35]:
for index, row in test_solutions.iterrows():
    new_test_df['id'].append(row['id'])
    new_test_df['task_id'].append(row['task_id'])
    new_test_df['student_solution'].append(row['student_solution'])
    new_test_df['author_comment'].append('')
    inp = final_test_df[final_test_df['task_id'] == row['task_id']]['input'].tolist()
    out = final_test_df[final_test_df['task_id'] == row['task_id']]['output'].tolist()
    new_test_df['input'].append(inp)
    new_test_df['output'].append(out)
    test_type = final_test_df[final_test_df['task_id'] == row['task_id']]['type'].tolist()
    new_test_df['type'].append(test_type)
    description = final_test_df[final_test_df['task_id'] == row['task_id']]['description'].tolist()[0]
    new_test_df['description'].append(description)
    author_solution = final_test_df[final_test_df['task_id'] == row['task_id']]['author_solution'].tolist()[0]
    new_test_df['author_solution'].append(author_solution)

In [36]:
new_test_df = pd.DataFrame(new_test_df)

In [37]:
new_test_df.shape

(325, 9)

In [38]:
new_test_df.sample(2)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type
207,499,31,Роман анализирует информаицю о переведенных те...,"[75000, 82100, 30000, 1000000]","[2004,французский,иврит,78918 слов\n1998,франц...",words = int(input())\n\nwith open('info_texts....,words = int(input())\n\nwith open('info_texts....,,"[open, closed, closed, closed]"
213,523,33,Игорь хочет пойти работать по профессии и смот...,[Организатор международных мероприятий: управл...,[Организатор международных мероприятий; Аккаун...,data = input().split('/ ')\nskill = input()\n\...,data = input().split('/ ')\nskill = input()\n\...,,"[open, open, open, closed, closed]"


In [39]:
final_test_prompt = []
for index, row in new_test_df.iterrows():
  description = row['description']
  res = []
  for i in range(len(row['input'])):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row['type'][i]} \n<Ввод>: {row['input'][i]} \n<Вывод>: {row['output'][i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: {author_comment}"
  final_test_prompt.append(prompt)

In [40]:
new_test_df['prompt'] = final_test_prompt

In [41]:
new_test_df.shape

(325, 10)

In [42]:
new_test_df.sample(2)

,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt
157,359,22,"Напишите функцию, определяющую возьмут ли диза...","[{\n 'AltAirResidence': (3000000, 'не реали...","[[42000000, False], [1000000, True], [160000, ...",def work(slov):\n counter = 0\n money_do...,def work(slov):\n counter = 0\n money_do...,,"[open, open, open, closed, closed, closed, clo...","<Task description>: Напишите функцию, определя..."
123,309,19,Ксения анализирует реализованные дизайн-проект...,"[20, 40, 8, 15, 43]","[2019,178 тыс. руб.,9 недель\n2022,180 тыс. ру...",week = int(input())\n\nwith open('projects_fil...,week = int(input())\n\nwith open('projects_fil...,,"[open, closed, closed, closed, closed]",<Task description>: Ксения анализирует реализо...


In [43]:
print(new_test_df['prompt'][0])

<Task description>: Реализуйте программу, которая напечатает стоимость реализации проекта со скидкой: 

В переменную dicount считывается размер скидки на реализацию проекта (вещественное число). Этот код уже написан.
В переменную money считывается стоимость реализации проекта в тысячах (целое число). Этот код уже написан.
Программа должна вывести фразу вида 'Реализация проекта будет стоить <стоимость> тыс. руб. без скидки. Со скидой стоимость составит <стоимость с учетом скидки> тыс. руб.'. 
Важно! В коде обязательно нужно использовать f-строку! 
<Example 1>: 
<Тип теста 1>: open 
<Ввод>: 0.1; 500 
<Вывод>: Реализация проекта будет стоить 500 тыс. руб. без скидки. Со скидкой стоимость составит 450.0 тыс. руб. 
<Example 2>: 
<Тип теста 2>: open 
<Ввод>: 0.05; 900000 
<Вывод>: Реализация проекта будет стоить 900000 тыс. руб. без скидки. Со скидкой стоимость составит 855000.0 тыс. руб. 
<Example 3>: 
<Тип теста 3>: open 
<Ввод>: 0; 2345678 
<Вывод>: Реализация проекта будет стоить 2345678

In [44]:
new_test_df.to_csv('new_test_df_for_inference.csv')

In [45]:
#####################################

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [46]:
system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. Write directly where the mistake is. Do not write a long answer. Provide only a hint that guides the student in the right direction, without giving away the full solution. People should not guess that you are an AI. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.
Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

prompt = new_test_df['prompt'][0]

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
]
answer = pipe(messages, max_length=4096, max_new_tokens=200)[0]['generated_text'][-1]['content']
print(answer)

Both `max_new_tokens` (=200) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<Комментарий эксперта>: В вашем решении есть небольшая ошибка в расчете стоимости со скидкой. Вы вычитаете из исходной стоимости произведение стоимости на скидку, что неверно. Правильно будет умножить исходную стоимость на (1 - скидку), чтобы получить стоимость со скидкой.


In [47]:
from tqdm import tqdm

In [48]:
# TEST inference

system_prompt = """

###INSTRUCTIONS###

You MUST ALWAYS:
- Answer in russian language
- I have no fingers and the placeholders trauma. NEVER use placeholders or omit the code
- You will be PENALIZED for wrong answers
- NEVER HALLUCINATE
- You DENIED to overlook the critical context
- ALWAYS follow ###Answering rules###

###Answering Rules###

Follow in the strict order:

1. USE the language of my message
2. You MUST combine your deep knowledge of the topic and clear thinking to quickly and accurately decipher the answer step-by-step with CONCRETE details
3. I'm going to tip $1,000,000 for the best reply
4. Your answer is critical for my career
5. Answer the question in a natural, human-like manner

You are a world-famous expert in teaching, known for your ability to deeply understand your students' thought processes and guide them toward the correct solutions. Your role is to receive solutions to problems from a student, analyze their approach, and provide thoughtful, personalized feedback. Your focus is on helping the student understand where they might have made a mistake or overlooked something without giving away the correct answer directly. Write directly where the mistake is. Do not write a long answer. Provide only a hint that guides the student in the right direction, without giving away the full solution. People should not guess that you are an AI. We will break the problem into smaller steps and focus on understanding each part by comparing the student's solution with the ideal one.

###CHAIN OF THOUGHT###

1. Clarify the Task: Restate the problem in simpler terms, focusing on what the student is trying to achieve.
2. Compare the Solutions: Directly compare the student's solution with the ideal solution:
   - Step 1: Analyze the input handling in both solutions. Are there any differences?
   - Step 2: Compare how conditions are checked in the student’s code versus the ideal solution. What differences do you observe?
   - Step 3: Identify specific points where the student’s logic deviates from the ideal solution.
3. Spot the Issue: Pinpoint where the logic in the student’s solution is incorrect:
   - {Provide a hint about the nature of the error without giving away the solution}.
4. Suggest an Improvement: Offer guidance on how to adjust the code to match the logic of the ideal solution, emphasizing where changes should be made.

Remember, making mistakes is part of learning. Think about each step carefully, and let’s find the best way to improve.
Example:
(Your task is to respond just as concisely and without giving the exact solution, like after <Expert's Comment>:): <Task description>: Реализуйте программу, которая напечатает хэштеги из фраз для лендинга сайта: \n\nВ переменную logo считывается текст лендинга. Этот код уже написан.\nПрограмма проверяет, есть ли в этом тексте хэштеги.  Хэштегами можно считать все элементы, которые начинаются со знака "#" и которые не состоят только из цифр (не включая знак "#"). Гарантируется, что хэштеги отделены от другого текста пробелом. \nВ конце программа печатает все хэштеги через запятую с пробелом. Хэштеги выводятся в том же порядке, в котором они идут в тексте.\nЕсли знаков хэштегов в тексте нет, то программа ничего не печатает. \n<Example 1>: \n<Тип теста 1>: open \n<Ввод>: Выучите китайский язык без усилий выполняя задания в приложении ( #the_best_app ) за #20 дней. \n<Вывод>: #the_best_app \n<Example 2>: \n<Тип теста 2>: open \n<Ввод>: Ваш вопрос ( #question ) - наш ответ ( #answer ) ! Мы всегда с Вами yf #100 % \n<Вывод>: #question, #answer \n<Example 3>: \n<Тип теста 3>: open \n<Ввод>: Мощный рывок для Вашего @бизнеса \n<Вывод>: nan \n<Example 4>: \n<Тип теста 4>: closed \n<Ввод>: Я никогда не был #счастье1 #любовь123 #123_123 qwe#qwe \n<Вывод>: #счастье1, #любовь123, #123_123 \n<Example 5>: \n<Тип теста 5>: closed \n<Ввод>: 9 из 10 экспертов рекомендуют # наш #_1223 продукт #__ . \n<Вывод>: #, #_1223, #__ \n<Example 6>: \n<Тип теста 6>: closed \n<Ввод>: # #123 #qswr_1 @qws1 $1#qwd 1#123_qwe #0 #____ #$567123 \n<Вывод>: #, #qswr_1, #____, #$567123 \n<Решение студента>: logo = input()\n\nres = []\nfor info in logo.split:\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n<Идеальное решение>: logo = input()\n\nres = []\nfor info in logo.split():\n    if info.startswith(\'#\') and info[1:].isdigit() == False:      \n        res.append(info)\n        \nprint(*res, sep=\', \') \n\n\n<Комментарий эксперта>: Вы забыли поставить скобки в методе .split().

"""

answers_test = []
for index, row in tqdm(new_test_df.iloc[:5].iterrows()):
  prompt = row['prompt']
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
  ]
  answer = pipe(messages, max_length=6048)[0]['generated_text'][-1]['content']
  answers_test.append(answer)

5it [01:38, 19.76s/it]


In [49]:
len(answers_test)

5

In [50]:
answer_test = pd.DataFrame(answers_test)

In [51]:
answer_test.sample(2)

,0
2,<Комментарий эксперта>: В вашем решении есть н...
1,<Комментарий эксперта>: Вы не закрыли скобку в...


In [52]:
for index, row in answer_test.iterrows():
    print(f'{index}) ', row[0])

0)  <Комментарий эксперта>: В вашем решении есть небольшая ошибка в расчете стоимости со скидкой. Вы вычитаете из исходной стоимости произведение стоимости на скидку, что неверно. Правильно будет умножить исходную стоимость на (1 - скидку), чтобы получить стоимость со скидкой.
1)  <Комментарий эксперта>: Вы не закрыли скобку в конце строки вывода, что приведет к ошибке. Кроме того, в вычислении стоимости с учетом скидки вы использовали неверную операцию. Вместо вычитания нужно умножить стоимость проекта на скидку и вычесть полученное значение из исходной стоимости. Также стоит добавить форматирование числа с двумя знаками после запятой для точности.
2)  <Комментарий эксперта>: В вашем решении есть небольшая ошибка в вычислении стоимости проекта со скидкой. Вы вычитаете из стоимости проекта произведение стоимости на скидку, что неверно. Вместо этого нужно умножить стоимость проекта на (1 минус скидку), чтобы получить стоимость со скидкой. Исправьте эту часть кода, и ваше решение будет п

## Saving the model

In [54]:
model.save_pretrained('20_steps_dpo_model')

## FINAL INFERENCE

In [ ]:
from typing import Callable

import pandas as pd
import torch
from transformers import BertModel, BertTokenizer

print("Loading models...", end="")
model_name = "DeepPavlov/rubert-base-cased-sentence"
emb_tokenizer = BertTokenizer.from_pretrained(model_name)
emb_model = BertModel.from_pretrained(model_name)
print("OK")


def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = emb_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = emb_model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding


def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])


def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])


def generate_submit(test_solutions: str, predict_func: Callable, save_path: str, use_tqdm: bool = True) -> None:
    bar = range(len(test_solutions))
    if use_tqdm:
        import tqdm

        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.id.iloc[i]
        solution_row = test_solutions.prompt.iloc[i]
        text = predict_func(solution_row)  # here you can do absolute whatever you want
        embedding = embedding2string(get_sentence_embedding(text))
        submit_df.loc[i] = [idx, text, embedding]
    submit_df.to_csv(save_path, index=False)

In [ ]:
def predict(prompt):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt},
    ]
    answer = pipe(messages, max_length=6048)[0]['generated_text'][-1]['content']
    #if '<Комментарий эксперта>:' in answer:
    #    answer = answer[24:] ########################
    return answer

In [ ]:
generate_submit(
        test_solutions=new_test_df,
        predict_func=predict,
        save_path="submit5_emae.csv",
        use_tqdm=True,
)

In [ ]:
df = pd.read_csv(path + "submit5_emae.csv")
df.head()